In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
#this combines the files together in the directories
import pathlib2 as pl2

def combine_files_together():
    base_path = "data/"
    elms_path = "ELMS"
    lmc_cup_path = "LeMansCup"
    fiawec_path = "FIAWEC"
    imsa_path = "IMSA"

    file_paths = [fiawec_path, elms_path, lmc_cup_path, imsa_path]
    #file_paths = [lmc_cup_path]
    #combine the dfs together
    for f in file_paths:
        curr_dir = base_path + f
        full_df = pd.DataFrame()
        current_circuit = ""
        ps = pl2.Path(curr_dir)
        rows = (pd.read_csv(p, index_col=0, dtype=str) for p in ps.glob('*.csv'))
        full_df = pd.concat(rows)
        full_csv_loc = f + "_full_data.csv"
        full_df = full_df.dropna(axis=1, how = "all")
        full_df = full_df.reset_index(drop=True)
        full_df.columns = full_df.columns.str.strip()
        full_df.columns = full_df.columns.str.lower()
        full_df.to_csv(full_csv_loc)
combine_files_together()

In [ ]:
#load all the combined dfs
def load_all_dfs():
    fia_wec_df = pd.read_csv('FIAWEC_full_data.csv', index_col = 0)
    imsa_df = pd.read_csv('IMSA_full_data.csv', index_col = 0,dtype=str)
    elms_df = pd.read_csv('ELMS_full_data.csv', index_col = 0)
    lmc_df = pd.read_csv('LeMansCup_full_data.csv', index_col = 0)
    full_df = pd.concat([fia_wec_df, imsa_df, elms_df, lmc_df])
    full_df = full_df.reset_index(drop=True)
    return full_df
full_df = load_all_dfs()


In [ ]:
full_df['key'] = full_df['season'].map(str) + "_" + full_df['championship'] + "_" + full_df['round'].map(str) \
        + "_" + full_df['session_type'] + "_" + \
                full_df['number'].map(str) +  "_" + \
                full_df['lap_number'].map(str)
full_df[full_df['number'] == '01']

In [ ]:
full_df = full_df.drop_duplicates()

In [ ]:
df_counts = full_df.groupby('key').count()
df_counts[df_counts['number'] > 1]

In [ ]:
#these help fix the team, group, class
full_df['fix_team'] = np.where(full_df['team'].isna(), np.where(full_df['team_x'].isna(), full_df['team_y'], full_df['team_x']),full_df['team'])
full_df[full_df['fix_team'].isna()]
full_df['fix_class'] = np.where(full_df['class'].isna(), np.where(full_df['class_x'].isna(), full_df['class_y'], full_df['class_x']),full_df['class'])
full_df[full_df['fix_class'].isna()]
full_df['fix_group'] = np.where(full_df['group'].isna(), np.where(full_df['group_x'].isna(), full_df['group_y'], full_df['group_x']),full_df['group'])
full_df[full_df['fix_group'].isna()]

full_df['team'] = full_df['fix_team']
full_df['class'] = full_df['fix_class']
full_df['group'] = full_df['fix_group']

full_df_fixed_columns = full_df.drop(['team_x', 'team_y', 'fix_team', 'group_x', 'group_y', 'fix_group', 'class_x', 'class_y','fix_class'], axis=1)
full_df_fixed_columns.columns

In [ ]:
#okay so the next thing to prob fix to drop porsche,ford,t6,t7, not used really for anything
#also fix s1,s2,s3, pit_time and convert to seconds
full_df_fixed_columns.info()

In [ ]:
#drop the ones we don't need like the le mans specific ones
full_df_without_lm = full_df_fixed_columns.drop(['porsche', 'ford', 't6', 't7'], axis=1)

#here we're going to fix the sectors, this is just showing why we would need to fix those
wanted_columns = ['s1', 's2', 's3', 'pit_time', 'elapsed',  's1_seconds', 's2_seconds', 's3_seconds', 's1_large', 's2_large', 's3_large']
full_df_sectors = full_df_without_lm[wanted_columns]
full_df_sectors.head(10)

In [ ]:
full_df_without_large = full_df_without_lm.drop(['s1_large', 's2_large', 's3_large'], axis=1)
full_df_without_large.info()

In [ ]:
#this we use to convert anything to ms
def convert_to_s(x):
    if(x == np.nan):
        return x
    hour = 0
    minutes = 0
    try:
        hour, minutes, second = x.split(":")
    except ValueError:
        hour = 0
        try:
            minutes, second = x.split(":")
        except ValueError:
            hour = 0
            minutes = 0
            second = x
    except AttributeError:
        print(x)
    hour_seconds = int(hour)*60*60
    if hour == 0 and minutes == 0:
        return x
    else:
        ms = 0
        try:
            seconds, ms = second.split(".")
        except ValueError:
            seconds = second
    minute_seconds = (int(minutes)*60 + int(seconds) + int(hour_seconds))*1000 
    total_ms = minute_seconds + int(ms)
    return total_ms/1000

#this uses the method to convert all of the large ones (h:m:s.ms to seconds)
def convert_df_cols_to_seconds(full_df):
    full_df['lap_time_seconds'] = full_df.lap_time.map(convert_to_s)
    full_df['s1_seconds'] = full_df.s1.map(convert_to_s)
    full_df['s2_seconds'] = full_df.s2.map(convert_to_s)
    full_df['s3_seconds'] = full_df.s3.map(convert_to_s)
    full_df['elapsed_seconds'] = full_df.elapsed.map(convert_to_s)
    full_df['pit_time_seconds'] = full_df.pit_time.map(convert_to_s)
    return full_df
full_df_fix_seconds = convert_df_cols_to_seconds(full_df_without_large)
full_df_fix_seconds 

In [ ]:
full_df_fix_seconds['session'] = full_df_fix_seconds['session_type']
full_df_fix_seconds

In [ ]:
def map_sessions(s):
    if ('QUALIFYING' in s):
        if ('RACE' in s):
            return 'RACE'
        return 'QUALIFYING'
    elif('SESSION' in s or 'TEST' in s):
        return 'TEST'
    elif ('PRACTICE' in s):
        return 'FREE PRACTICE'
    elif ('RACE' in s):
        return 'RACE'
    else:
        return s
#maps the sessions to session_types

full_df_fix_seconds['session_type'] = full_df_fix_seconds['session'].map(map_sessions)
full_df_test = full_df_fix_seconds.groupby(['session','session_type']).mean()
full_df_test

In [ ]:
full_df_fix_seconds.info()

In [ ]:
full_df_fix_seconds[(full_df_fix_seconds['team'].isna())]

In [ ]:
full_df_fix_seconds_columns = full_df_fix_seconds.drop(['s1', 's2', 's3', 'lap_time', 'elapsed', 'pit_time'], axis=1)

In [ ]:
full_df_fix_seconds_columns.to_csv('alkamel_data_full.csv', index=False)

In [ ]:
full_df_test['event'] = full_df_test['circuit']

In [ ]:
full_df_test.to_csv('new_2022_data.csv')

In [ ]:
full_df[full_df['team'].isna()]

In [ ]:
full_df.to_csv('final_2022_data.csv', index=False)

In [ ]:
wanted_columns = ['event', 'circuit']
full_df['event'] = full_df['circuit']
full_df_group = full_df.groupby(['circuit','event']).mean().reset_index()
full_df_group[wanted_columns].to_csv('new_2022_events.csv',index=False)

In [ ]:
full_df = full_df.drop('event', axis=1)

In [ ]:
events = pd.read_csv('new_2022_events.csv')
full_df_w_event = pd.merge(left = full_df, right = events, on = 'circuit')

In [ ]:
full_df_w_event['event'] = full_df_w_event['event_x']
full_df_w_event['circuit'] = full_df_w_event['event_y']
full_df_w_event = full_df_w_event.drop(['event_x', 'event_y','s1', 's2', 's3', 'hour', 'elapsed', 'lap_time'], axis=1)

In [ ]:
full_df_w_event.groupby(['circuit','event']).mean()

In [ ]:
full_df_w_event.info()

In [ ]:
def remap_columns():
    full_df_w_event['driver_number'] = full_df_w_event['driver_number'].map(int)
    full_df_w_event['lap_number'] = full_df_w_event['lap_number'].map(int)
    full_df_w_event['lap_improvement'] = full_df_w_event['lap_improvement'].map(int)
    full_df_w_event['s1_improvement'] = full_df_w_event['s1_improvement'].map(int)
    full_df_w_event['s2_improvement'] = full_df_w_event['s2_improvement'].map(int)
    full_df_w_event['s3_improvement'] = full_df_w_event['s3_improvement'].map(int)
    full_df_w_event['kph'] = full_df_w_event['kph'].map(float)
    full_df_w_event['top_speed'] = full_df_w_event['top_speed'].map(float)
    full_df_w_event['round'] = full_df_w_event['round'].map(int)
    full_df_w_event['lap_time_seconds'] = full_df_w_event['driver_number'].map(float)
    full_df_w_event['s1_seconds'] = full_df_w_event['s1_seconds'].map(float)
    full_df_w_event['s2_seconds'] = full_df_w_event['s2_seconds'].map(float)
    full_df_w_event['s3_seconds'] = full_df_w_event['s3_seconds'].map(float)
    full_df_w_event['elapsed_seconds'] = full_df_w_event['elapsed_seconds'].map(float)
    full_df_w_event['pit_time_seconds'] = full_df_w_event['pit_time_seconds'].map(float)
full_df_w_event.info()

In [ ]:
full_df_w_event.to_csv('2022_data_final.csv')

In [ ]:
full_df_w_event.columns